In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import GenericUnivariateSelect, chi2, mutual_info_classif
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from scipy.sparse import hstack, vstack
from pickle import dump,load

In [2]:
features = pd.read_pickle("/home/binoy/OneDrive/UH/COSC 6342 - Machine Learning/Project/GenderClassification/data/feature_dumps/scaled_features.pkl")

In [3]:
features = features.drop(columns=['NN', 'NNPS', 'VBD', 'VBZ', 'MD',
       'EX', 'IN', 'VB', 'JJR', 'JJS', 'PRP', 'WDT', 'JJ', 'VBP', 'NNS',
       'VBN', 'DT', 'RB', 'WP', 'VBG', 'NNP', 'RBR', 'PRP$', 'JJ NN',
       'VBP VB', 'VBD PRP', 'IN IN', 'NNP VBZ', 'RB DT', 'NN VBG',
       'IN JJ', 'NN NN', 'RB VBZ', 'VBG DT', 'NN NNS', 'VBZ JJ', 'IN RB',
       'JJ JJ', 'NN VBZ', 'IN VBG', 'VBP DT', 'VB NN', 'NNS VBP',
       'DT NNP', 'PRP VBZ', 'PRP VBD', 'PRP VB', 'NN PRP', 'NN DT',
       'VBZ VB', 'PRP VBP', 'PRP$ JJ', 'VBD IN', 'VB JJ', 'NN JJ',
       'RB VB', 'JJ NNP', 'RB VBG', 'VBZ PRP', 'VBD DT', 'RB RB', 'JJ VB',
       'PRP RB', 'JJ IN', 'VBD VB', 'VB IN', 'VBP PRP', 'VBD RB',
       'VBG IN', 'PRP IN', 'VB PRP', 'NN RB', 'NNP NN', 'VB VBN',
       'NN NNP', 'IN NN', 'VBP VBN', 'NN WDT', 'RB IN', 'DT JJ', 'RB VBD',
       'VBZ IN', 'NN MD', 'VB DT', 'NNS IN', 'NNP RB', 'VB PRP$',
       'VBP IN', 'RB VBP', 'NNS RB', 'DT NN', 'VBZ VBN', 'MD RB',
       'NNP IN', 'NN VBD', 'JJ NNS', 'NN VB', 'IN PRP$', 'MD VB',
       'RB PRP', 'NNS VB', 'VBZ DT', 'VBG NN', 'VBN IN', 'PRP$ NNS',
       'VB VB', 'VBP RB', 'NNP NNP', 'NN IN', 'VB RB', 'VBG PRP',
       'PRP MD', 'IN DT', 'NNP VBD', 'IN NNS', 'IN NNP', 'RB JJ',
       'IN PRP', 'VBD JJ', 'RB VBN', 'DT NNS', 'VBD VBN', 'PRP$ NN',
       'VBZ RB', 'VBP JJ', 'DT NN VBZ', 'PRP VBD IN', 'JJ NNP NN',
       'DT NNP NNP', 'RB IN PRP', 'IN JJ NN', 'JJ NN VB', 'IN NN IN',
       'IN DT NNS', 'NNS IN PRP', 'NN IN NNS', 'VBN IN DT', 'PRP VBP JJ',
       'DT NNP NN', 'VBZ DT NN', 'PRP VBP RB', 'IN DT JJ', 'MD RB VB',
       'JJ NN IN', 'RB DT NN', 'VBG DT NN', 'NNP NN NN', 'DT NNS IN',
       'DT NN IN', 'JJ NN NN', 'VB DT NN', 'DT JJ NNS', 'NN IN PRP$',
       'IN JJ NNS', 'NNS IN NN', 'VBZ DT JJ', 'PRP VBP DT', 'NNP NNP NN',
       'PRP VBP PRP', 'VB DT JJ', 'VB PRP$ NN', 'DT JJ JJ', 'IN PRP VBP',
       'DT NN VB', 'IN NNP NNP', 'NNS IN DT', 'VB IN DT', 'IN DT NNP',
       'NN IN NNP', 'IN PRP$ NN', 'DT JJ NN', 'NN NN IN', 'IN NN NN',
       'VBP DT NN', 'PRP MD VB', 'PRP$ NN NN', 'RB JJ NN', 'DT NN NN',
       'IN PRP$ JJ', 'VBD DT JJ', 'PRP VBP VB', 'NN NN NN', 'VBD DT NN',
       'NNP NNP NNP', 'PRP VBD VB', 'JJ NNS IN', 'IN DT NN', 'NN IN NN',
       'PRP VBP IN', 'PRP VBD DT', 'IN PRP VBD', 'NN IN JJ', 'DT NN RB',
       'NN IN PRP', 'NN IN DT', 'RB IN DT', 'JJ JJ NN', 'PRP$ NN IN',
       'RB PRP VBP', 'PRP$ JJ NN', 'IN DT JJ NN', 'NN IN DT NN',
       'NN IN PRP$ NN', 'NN IN DT JJ', 'IN DT NN VB', 'DT JJ NN IN',
       'IN DT NN NN', 'DT JJ NN NN', 'VB DT JJ NN', 'IN DT NNP NN',
       'DT NN IN PRP', 'JJ NN IN DT', 'DT NN IN NN', 'NNS IN DT NN',
       'VB DT NN IN', 'NN IN JJ NN', 'DT NN IN DT', 'JJ NN IN NN',
       'IN DT NN IN', 'DT NN IN DT NN', 'NN IN DT JJ NN',
       'DT JJ NN IN NN', 'JJ NN IN DT NN', 'DT VBZ', 'DT RB', 'VBD NN',
       'NNP VB', 'IN NNP NN', 'JJ NNS VB', 'PRP$ NN VB', 'VBN IN NN',
       'NN NN VB', 'PRP RB VB', 'VBZ NN', 'DT IN', 'DT VB', 'RB NN',
       'WP VB', 'VBP NN', 'DT DT', 'VBN NN', 'IN VB', 'WDT VB',
       'IN PRP VB', 'NNP NNP VB', 'NN IN VB', 'RB PRP VB', 'NN PRP VB'])

In [3]:
gender = features.Gender
features = features.drop(columns=['Gender'])

In [4]:
train_data, test_data, gender_train, gender_test = train_test_split(features, gender, test_size=0.1, shuffle=True, stratify=gender, random_state=42)

In [5]:
train_data, val_data, gender_train, gender_val = train_test_split(train_data, gender_train, test_size=0.22, shuffle=True, stratify=gender_train, random_state=42)

In [6]:
train_blog = train_data.Blog
train_pos = train_data.POS
val_blog = val_data.Blog
val_pos = val_data.POS
test_blog = test_data.Blog
test_pos = test_data.POS
train_data = train_data.drop(columns=['Blog', 'POS'])
val_data = val_data.drop(columns=['Blog', 'POS'])
test_data = test_data.drop(columns=['Blog', 'POS'])

In [8]:
nrange = (2, 8)
cv_blog_word = CountVectorizer(binary=True, ngram_range=nrange)
cv_blog_char = CountVectorizer(binary=True, ngram_range=nrange, analyzer='char')
cv_pos = CountVectorizer(binary=True, ngram_range=nrange)

In [9]:
train_blog_word_features = cv_blog_word.fit_transform(train_blog.astype(str))
train_blog_char_features = cv_blog_char.fit_transform(train_blog.astype(str))
train_pos_word_features = cv_pos.fit_transform(train_pos.astype(str))

In [10]:
val_blog_word_features = cv_blog_word.transform(val_blog.astype(str))
val_blog_char_features = cv_blog_char.transform(val_blog.astype(str))
val_pos_word_features = cv_pos.transform(val_pos.astype(str))
test_blog_word_features = cv_blog_word.transform(test_blog.astype(str))
test_blog_char_features = cv_blog_char.transform(test_blog.astype(str))
test_pos_word_features = cv_pos.transform(test_pos.astype(str))

In [11]:
train_features = hstack((train_blog_word_features, train_blog_char_features, train_pos_word_features, train_data), format='csr')
val_features = hstack((val_blog_word_features, val_blog_char_features, val_pos_word_features, val_data), format='csr')
test_features = hstack((test_blog_word_features, test_blog_char_features, test_pos_word_features, test_data), format='csr')

# train_features = hstack((train_blog_word_features, train_blog_char_features, train_data), format='csr')
# val_features = hstack((val_blog_word_features, val_blog_char_features, val_data), format='csr')
# test_features = hstack((test_blog_word_features, test_blog_char_features, test_data), format='csr')

In [12]:
n_features = 50000
transformer_chi = GenericUnivariateSelect(chi2, 'k_best', param=n_features)
# transformer_mi = GenericUnivariateSelect(mutual_info_classif, 'k_best', param=n_features)

In [13]:
train_features_chi = transformer_chi.fit_transform(train_features, gender_train)
val_features_chi = transformer_chi.transform(val_features)
test_features_chi = transformer_chi.transform(test_features)
# train_features_mi = transformer_mi.fit_transform(train_features, gender_train)
# val_features_mi = transformer_mi.transform(val_features)
# test_features_mi = transformer_mi.transform(test_features)

In [14]:
mlp_chi = MLPClassifier(hidden_layer_sizes=(75, 75), activation='identity', solver='adam', random_state=42, early_stopping=True, max_iter=2500)
# mlp_mi = MLPClassifier(hidden_layer_sizes=(75, 75), activation='identity', solver='adam', random_state=42, early_stopping=True, max_iter=2500)

In [15]:
mlp_chi.fit(train_features_chi, gender_train)
mlp_chi.score(val_features_chi, gender_val)

0.7232704402515723

In [16]:
# mlp_mi.fit(train_features_mi, gender_train)
# mlp_mi.score(val_features_mi, gender_val)

In [17]:
tvfeatures = vstack((train_features_chi, val_features_chi), format='csr')
tvgender = pd.concat([gender_train, gender_val], ignore_index=True)

# print(type(gender_train), type(gender_val), gender_train.shape, gender_val.shape)

In [11]:
mlp = MLPClassifier(hidden_layer_sizes=(75, 75), activation='identity', solver='adam', random_state=42, early_stopping=True, max_iter=2500)
# score = cross_val_score(mlp, tvfeatures, tvgender, cv=10, scoring='accuracy', n_jobs=2, verbose=15)
# print(score)
# print(np.mean(score))

In [19]:
mlp_chi.score(test_features_chi, gender_test)

0.6645962732919255

In [7]:
train_data, test_data, gender_train, gender_test = train_test_split(features, gender, test_size=0.1, shuffle=True, stratify=gender, random_state=42)

In [8]:
cv = CountVectorizer(binary=True, ngram_range=(2, 8))
td = cv.fit_transform(train_data.Blog.astype(str))

In [ ]:
score = cross_val_score(mlp, td, gender_train, cv=10, scoring='accuracy', n_jobs=2, verbose=15)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
score

In [ ]:
np.mean(score)